<a href="https://colab.research.google.com/github/mbaliu-treino/Desenvolve/blob/main/LEARN_C_NLP_Word2Vec_CBOW_e_Skipgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=orange><b>Word2Vec: interpretação da linguagem humana com Word embedding</b></font>

<font color=gray size=2><a href=https://colab.research.google.com/drive/1k1dHy27lP0iWt356_TNg37TyD4pRG43Z>Arquivo Colab</a></font>

<ul><font size=2 color=gray><b>FICHA TÉCNICA</b>
<li><a href=https://cursos.alura.com.br/course/introducao-word-embedding><font size=2 color=gray>Word2Vec: interpretação da linguagem humana com Word embedding</a>
<li>Carga Horária: 10 h
<li>Instrutora: Thiago G Santos
<li>Data de Início: 08-2022
</ul>

<hr color=gray><br>

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)


<h3><b>Conteúdo / Aprendizagem:</b></h3>

- 

# <font color=orange>RESUMO</font>

<a href=https://docs.google.com/spreadsheets/d/1tTygYlq9r7nkUsw9a25N5_z57-de_59tSBCIVPw6KUw><font size=2 color=gray>ROTEIROS Data Science</font></a>

* <font color=orange><b>

# <font color=orange>CURSO</font>

**Projeto**: criar um classificador do gênero de nóticias de jornal através de seus títulos.



1. Vetorização de palavras: 
    - One-Hot-Encoding (Vetor esparso)
    - Word2Vec: vetor denso de dimensão fixa

Generalização dos contextos das palavras. Esta técnica de vetorização permite operar com a informação do contexto das palavras.

**CBOW** - Continous back of words

`Moro ____ país.`

**SKIP GRAM**

`____ França ____`

<a href=https://github.com/alura-cursos/word2vec/blob/aula6/Aula_6.ipynb><font size=2; color=gray>Material de referencial do curso</a></font>

## <font color=orange>Dados</font>

A base de dados será a base de conhecimento para o modelo, fornecendo um conhecimento previamente já consolidado que servirá para o treinamento. No NLP, essa base de dados é conhecimento como *corpus textual*.



In [ ]:
# Para o output do Colab quebrar as linhas longas
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import pandas as pd

In [ ]:
uri_treino = 'https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/treino.csv'
uri_teste = 'https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/teste.csv'

In [ ]:
# Leitura dos dados
artigo_treino = pd.read_csv(uri_treino)
artigo_teste = pd.read_csv(uri_teste)

In [ ]:
# Formato da tabela
print(artigo_treino.shape)
print(artigo_teste.shape)

(90000, 6)
(20513, 6)


In [ ]:
# Colunas
print(*artigo_treino.columns, sep=', ')

title, text, date, category, subcategory, link


In [ ]:
# Exemplos de registros
artigo_treino.sample(3)

,title,text,date,category,subcategory,link
56823,Saiba quem são as vítimas dos atentados terror...,"A Presidência da França anunciou, nesta quarta...",2015-11-15,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/11/170...
6749,"Surfista a ser batido em 2015, Medina inicia M...","Há um ano, Gabriel Medina era um jovem muito p...",2015-02-27,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/02/1...
62676,Música em Letras: Será que a casa caiu?,"Estabelecida na Rua Padre Cerda, 25, no Alto d...",2015-02-02,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/02...


In [ ]:
# Exemplo de título
artigo_teste.iloc[643]['title']

'Somos todos imprevisíveis'

# <font color=orange>**Vetorização**</font>

Vetorização de textos


## <font color=orange>One Hot Encoding</font>

Uma forma de representar uma palavra de forma matemática é usar a técnica de vetores. Uma das mais conhecidas é a One-Hot-Encoding que transforma palavras em um vetor binário. Ela é muito usada para tranformar variáveis multinomiais em valores numéricos.

Cada palavra se tornará uma dimensão no vetor. Logo a quantidade de dimensões será igual ao tamanho do <font color=orange>**vocabulário**</font>.

O produto dessa transformação é uma <font color=orange>**matriz esparsa**</font>.

Um problema do One-Hot-Encode é que ele torna cada token desconectada, sem contexto ou referências.

Como consequencia teremos muitos dados sem dados relevantes, ou seja, um monte de zeros.

* Tabela esparsa (monte de zeros)
* Tamanho equivalente à quantidade de classes (palavras)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vetorizador = CountVectorizer()

In [ ]:
# Teste de vetorização 
texto_exemplo = ['tenha um bom dia',
                 'tenha um péssimo dia',
                 'tenha um ótimo dia']

vetorizador.fit(texto_exemplo)

CountVectorizer()

In [ ]:
# Vocabulário
vetorizador.vocabulary_

{'tenha': 3, 'um': 4, 'bom': 0, 'dia': 1, 'péssimo': 2, 'ótimo': 5}

In [ ]:
vetor_teste = vetorizador.transform(['bom', 'um'])
# Posição na matriz esparsa
print('Posição na matriz esparsa:\n', vetor_teste)

# Vetor de matriz esparsa
print('\nMatriz esparsa:\n', vetor_teste.toarray())

Posição na matriz esparsa:
   (0, 0)	1
  (1, 4)	1

Matriz esparsa:
 [[1 0 0 0 0 0]
 [0 0 0 0 1 0]]


## <font color=orange>Word2Vec</font>

Uma alternativa ao sistema de vetorização One-Hot Encoding com a matriz esparsa é o método Word2Vec que permite a criação de uma **tabela densa de tamanho fixo**.

Esse novo vetor é do tipo denso, em que todas as suas dimensões possui um valor numérico.

* Vetor denso
* Tamanho fixo

Um dos objetivos do Word2Vec é conseguir ensinar o <font color=orange>**CONTEXTO**</font> para as máquinas. Dando um sentido para as palavras e consequentemente conseguir identificar grupos semelhantes a eles. 

No exemplo do curso, o contexto das palavras a seguir é o que o nome se refere (nome de carro ou nomes de países). Além disso, há a possibilidade de sub-grupos nos grupos, criando mais relações e contextos.

Para isso serão necessários uma grande quantidade de dados, para fazer um treinamento semelhante à de redes neurais. Com o treinamento extensivo é possível extrair o uso do contexto das palavras.

A saída do processamento com o Word2Vec é um vetor denso, onde todas as posições possui um valor numérico.

![Word2Vec](https://lh3.googleusercontent.com/gmRC2djenrwMuKAe-CahjvIplP-gFfAmArlcbJaTwzXJIYMVAUpiiQX8kcEFt0E9prpUd6koomZpC7IXEOWoxtamNgeQGXG7snDIv_si2vPEQhr2eTri3H5en11fy8qPiAQwjWUc0TsMuLw1hjHJZqs)

### <font color=orange>CONTEXTO</font>

<h3>Mas como é possível ensinar as máquinas a aprender o contexto?</h3>

* <font color=orange>DADOS</font>: Quanto maior a quantidade de dados, melhor será a representação vetorial do Word2Vec.
* <font color=orange>TREINAMENTO</font>: tendo um conjunto de dados de entrada e uma saída de número fixo, que representarão os pesos, podemos fazer o treinamento com redes neurais.



Para interpretar o contexto, somente a palavra anterior e a seguinte serão usadas.

TREINAMENTO

`Prever a palavra através do contexto`

Com as palavras adjacentes, ou seja, o contexto da palavra, para tentar adivinhar qual será a nossa saída. Com isso, podemos ajustar pesos das fuções das camadas ocultas da rede neural.

0. Remoção de Stopwords
1. ENTRADA: As palavras de contexto serão a entrada do modelo.
2. CAMADAS OCULTAS: processamento matemático de inúmeras funções.
3. SAÍDA: o conjunto de palavras esperados.

Essa técnica de a partir das palavras de contexto tentar adivinhar a palavra ser preenchida é conhecida como CBOW (Continuous Bag of Words)

* Processo de treinamento mais rápido.

TREINAMENTO 2

`Prever o CONTEXTO através da palavra`

Neste modelo temos uma palavra e gostaríamos de prever o que vem antes e o que vem depois.

1. ENTRADA: uma palavra
2. CAMADAS OCULTAS
3. SAÍDA: n palavra de contexto

Esta técnica é conhecida como **Skip Gram**. 

* melhor contextualização, com volumes de dados menores

> "O CBOW é treinado para realizar a previsão de uma determinada palavra tendo o contexto como informação de entrada, já o Skip-Gram precisa prever o contexto recebendo como input uma palavra. Ambos geram um vetor denso de tamanho predeterminado."

Com a comparação dos contextos de forma matemática, o modelo será capaz de posicionar os dados de próximas ou mais distantes, usando o contexto como elemento separador.

## Utilização do Word2Vec Pre-treinado

1. Buscar uma fonte confiável
  * NILC - Núcleo Institucional de Linguística Computacional da USP

O contéudo do modelo treinado fornecido traz o token e depois 300 dimensões com valores como saída da rede neural.

palavra -> [x, y, ..., z]

In [ ]:
# Download de tokens já construído
uri_cbow_300 = 'http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip'

!wget $uri_cbow_300
!unzip "/content/download.php?file=embeddings%2Fword2vec%2Fcbow_s300.zip"

--2022-09-08 21:20:08--  http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip
Connecting to 143.107.183.175:22980... connected.
HTTP request sent, awaiting response... 200 OK
Length: 929305948 (886M) [application/octet-stream]
Saving to: ‘download.php?file=embeddings%2Fword2vec%2Fcbow_s300.zip’

download.php?file=e 100%[===================>] 886.25M  11.2MB/s    in 80s     

2022-09-08 21:21:29 (11.1 MB/s) - ‘download.php?file=embeddings%2Fword2vec%2Fcbow_s300.zip’ saved [929305948/929305948]

Archive:  /content/download.php?file=embeddings%2Fword2vec%2Fcbow_s300.zip
  inflating: cbow_s300.txt           


In [ ]:
# Leitura do arquivo de tokens
cbow_300_path = '/content/cbow_s300.txt'

with open(cbow_300_path) as file:
  for linha in range(10):
    print(next(file))

929606 300

</s> -0.001667 -0.000158 -0.000026 0.001300 -0.000796 0.001527 0.000046 0.000584 0.000449 -0.000100 0.000353 0.001251 0.001069 0.000506 0.000574 0.000838 -0.000930 -0.001220 0.000317 0.001315 -0.001120 0.001373 -0.000040 -0.001580 0.000421 -0.000667 -0.001556 -0.000746 0.001604 0.001157 -0.000027 0.000354 0.000358 -0.000527 -0.000573 -0.001512 -0.001557 -0.001637 0.001617 -0.001511 -0.001022 -0.001426 0.001086 -0.001033 0.000593 0.000724 0.000627 -0.000450 -0.001140 0.000333 0.000524 0.001541 0.000284 0.000617 -0.000807 -0.000088 -0.000364 0.001126 -0.001230 -0.001138 -0.001280 0.001330 0.001257 0.000576 0.000764 0.000684 0.001008 -0.000215 -0.000629 -0.001228 -0.001557 -0.000311 -0.000246 0.000045 0.001136 -0.000645 -0.000549 0.001099 0.000858 -0.000886 0.000553 0.000303 0.001433 0.000732 0.001321 -0.000894 -0.000700 -0.000661 -0.001484 -0.000950 -0.001556 -0.000809 0.000348 -0.000068 0.000724 -0.000569 -0.000161 -0.001628 -0.001437 -0.000259 -0.000296 -0.001571 0.000149 0

In [ ]:
# Construção do modelo Word2Vec a partir do texto ~2 min 44 s
from gensim.models import KeyedVectors

wv_from_text = KeyedVectors.load_word2vec_format(cbow_300_path)
modelo = wv_from_text

In [ ]:
# Vetor de uma palavra
vetor_china = modelo.get_vector('china')
len(vetor_china)

300

In [ ]:
# Tokens mais próximos de outro token
modelo.most_similar('cocaína')

[('maconha', 0.7640054225921631),
 ('haxixe', 0.6427118182182312),
 ('heroína', 0.6224136352539062),
 ('crack', 0.6031304001808167),
 ('droga', 0.5509088635444641),
 ('marijuana', 0.5143439769744873),
 ('liamba', 0.510414183139801),
 ('ecstasy', 0.5004808902740479),
 ('morfina', 0.48937979340553284),
 ('metanfetamina', 0.4885428845882416)]

### Combinações - Embedding

In [ ]:
# COMBINAÇÃO: Tokens mais próximos de combinação de tokens
modelo.most_similar(positive=['brasil', 'argentina'])

[('chile', 0.6781662702560425),
 ('peru', 0.6348033547401428),
 ('venezuela', 0.6273865103721619),
 ('equador', 0.6037014722824097),
 ('bolívia', 0.6017140746116638),
 ('haiti', 0.5993807315826416),
 ('méxico', 0.5962306261062622),
 ('paraguai', 0.5957703590393066),
 ('uruguai', 0.5903672575950623),
 ('japão', 0.5893509984016418)]

In [ ]:
modelo.most_similar(positive=['frança', 'cocaína'])

[('espanha', 0.606521487236023),
 ('itália', 0.5986364483833313),
 ('inglaterra', 0.5887600183486938),
 ('maconha', 0.5859187841415405),
 ('áustria', 0.5843812227249146),
 ('suécia', 0.5743238925933838),
 ('grã-bretanha', 0.5708452463150024),
 ('holanda', 0.5653262734413147),
 ('dinamarca', 0.5606917142868042),
 ('heroína', 0.556676983833313)]

In [ ]:
modelo.most_similar(positive=['bahia', 'pará'])

[('ceará', 0.7064878940582275),
 ('paraíba', 0.6952986717224121),
 ('piauí', 0.6575934290885925),
 ('paraná', 0.6201068758964539),
 ('goiás', 0.6155080795288086),
 ('maranhão', 0.6058744192123413),
 ('tocantins', 0.6037553548812866),
 ('acre', 0.5950236916542053),
 ('amapá', 0.5813421010971069),
 ('alagoas', 0.5803781747817993)]

#### Plural

In [ ]:
# nuvens -> nuvem
# estrelas -> estrelas

# nuvens + estrela - nuvem = estrelas

modelo.most_similar(positive=['nuvens', 'estrela'], negative=['nuvem'])

[('estrelas', 0.5497429966926575),
 ('plêiades', 0.379197895526886),
 ('colinas', 0.3746805191040039),
 ('trovoadas', 0.373703271150589),
 ('sombras', 0.3734194040298462),
 ('pombas', 0.3726757764816284),
 ('corredoras', 0.3640727698802948),
 ('cigarras', 0.36065393686294556),
 ('galáxias', 0.35754913091659546),
 ('luas', 0.3575345277786255)]

#### Declinação de Gênero

In [ ]:
# médico + ela - ele = médica
modelo.most_similar(positive=['empresário', 'pesquisadora'], negative=['pesquisador'])

[('empresária', 0.5880690813064575),
 ('advogada', 0.5456726551055908),
 ('antropóloga', 0.5085045695304871),
 ('blogueira', 0.5056331157684326),
 ('psicóloga', 0.5037093162536621),
 ('bióloga', 0.48805463314056396),
 ('cabeleireira', 0.4795725345611572),
 ('aposentada', 0.47835773229599),
 ('pedagoga', 0.47171372175216675),
 ('diarista', 0.4525532126426697)]

In [ ]:
# médico + ela - ele = médica
modelo.most_similar(positive=['médico', 'pesquisadora'], negative=['pesquisador'])

[('psicóloga', 0.572142481803894),
 ('enfermeira', 0.5531742572784424),
 ('bióloga', 0.5138505697250366),
 ('antropóloga', 0.508258044719696),
 ('nutricionista', 0.49191129207611084),
 ('advogada', 0.489876925945282),
 ('socióloga', 0.47331202030181885),
 ('blogueira', 0.4691736102104187),
 ('cabeleireira', 0.46026867628097534),
 ('empresária', 0.44862642884254456)]

### Esteriótipos

O modelo absorde os esteriótipos da sociedade através das escritas realizadas pela sociedade, pois o modelo não realiza uma análise do sentido das palavras, mas sim o comportamento da escrita humana.

> Como discutimos em aula, a inteligência artificial é uma das áreas com maior potencial de transformar o modo como vivemos, da mobilidade à saúde. Com o impulso e progressos da IA nos últimos anos, foi possível o avanço tecnológico dos hardwares e do aumento exponencial de dados disponíveis para treinamento dos modelos de aprendizagem de máquina.

> Como os modelos mais promissores na área dependem muito dos dados que produzimos, esses modelos podem reproduzir estereótipos da nossa sociedade, impulsionando ainda mais a desigualdade social. Desenvolver modelos que não reproduzam os aspectos negativos da sociedade e importantíssimo para garantir um futuro mais justo a todos, por isso áreas de pesquisa como ética na IA vem crescendo consideravelmente nos últimos anos.

> Gostaria de deixar como recomendação para quem se interessar este [artigo](https://www.weforum.org/agenda/2019/01/ai-isn-t-dangerous-but-human-bias-is/) apresentado no encontro anual do Fórum Económico Mundial, escrito por Richard Socher, cientista chefe da Salesforce, que aborda de maneira muito interessante os perígos envolvidos na reprodutibilidade de estereótipos humanos em sistemas de IA.

> Outra ótima referência é o [capítulo 6, seção 6.11 - Bias and Embeddings](https://web.stanford.edu/~jurafsky/slp3/6.pdf), do livro Speech and Language Processing escrito por Dan Jurafsky e James H. Martin. Nesta seção eles abordam o tema viés, além de trazer referências com técnicas para reduzir este problema no embedding de palavras.

## Aplicação para o case

As notícias que desejamos classificar são um conjunto de palavras, então é preciso vetorizar essas notícias para que sejam processadas.

In [ ]:
# Análise das frases: possuem acentuação e pontuação
artigo_treino['title'].loc[12]

"Daniel Craig será stormtrooper em novo 'Star Wars', diz ator"

### Tokenização

O modelo de Word2Vec pre-treinado já passou por um pré-processamento, o que irá impactar no nossos tokens. 

Por exemplo, palavras raras, como palavras com erro de digitação foram transformadas para `UNKNOWN`. Números foram transformados para `0` ou `00...`. Urls para `URL`. Emails para `EMAIL`.

O tokenize tem dificuldade de transformar o primeiro aspas simples.

In [ ]:
# Tokenização - quebra por palavras
import nltk
import string
nltk.download('punkt')

def tokenizador(texto, language='pt'):
  texto = texto.lower()
  lista_alfanumerico = []
  pontuacao = string.punctuation

  for token in nltk.word_tokenize(texto):
    if token in pontuacao:
      continue

    if language == 'pt':
      lista_alfanumerico.append(token.replace("'", ''))

  return lista_alfanumerico

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
tokenizador("Daniel Craig será stormtrooper em novo 'Star Wars', diz ator")
# tokenizador("Yes, I'm right!")
# tokenizador('Ela me disse: "cara bicho!"')
# "   'Star Wars'".strip()

['daniel',
 'craig',
 'será',
 'stormtrooper',
 'em',
 'novo',
 'star',
 'wars',
 'diz',
 'ator']

### Combinando vetores

* Média dos vetores de cada palavra
* Soma
* Combinações com redes neurais

* [Técnicas de combinação de vetores](https://www.kaggle.com/c/quora-insincere-questions-classification/discussion/71778)

In [ ]:
import numpy as np

def combinacao_de_vetores_soma(palavras_numeros, word2vec_model):
  vetor_resultante = np.zeros(300)  # vetor esparso

  for pn in palavras_numeros:
    # para cada palavra será adicionado ao vetor combinado os valores do Word2Vec(300)
    try:
      vetor_resultante += word2vec_model.get_vector(pn)
    except KeyError:
      if pn.isnumeric():
        pn = '0' * len(pn)
        vetor_resultante += word2vec_model.get_vector(pn)
      elif '@' in pn:
        pn = 'EMAIL'
        vetor_resultante += word2vec_model.get_vector(pn)
      elif 'http' in pn:
        pn = 'URL'
        vetor_resultante += word2vec_model.get_vector(pn)
      else:
        pn = 'unknown'
        vetor_resultante += word2vec_model.get_vector(pn)

  return vetor_resultante

In [ ]:
palavras_numeros = tokenizador('texto 55 email@meu.com textosss')
vetor_texto = combinacao_de_vetores_soma(palavras_numeros, word2vec_model=modelo)
vetor_texto

array([ 0.71333102,  0.233138  , -0.262831  ,  0.40465299,  0.08968799,
       -0.12859901,  0.36422   ,  0.06705298, -0.42137901,  0.22912   ,
        0.20574899, -0.422199  , -1.00184998,  0.79917402,  0.083206  ,
        0.41008801, -1.40153898, -0.21416499, -0.62434299, -0.59793197,
       -0.105193  , -0.206048  ,  0.036963  ,  1.22250398, -0.40927802,
       -0.015125  , -0.61525903,  0.14933401,  0.22086101, -0.47592898,
       -0.65150899,  0.059235  ,  0.380605  , -0.31868899,  0.46247097,
        0.409421  ,  0.84811502,  0.25158099,  0.37866702,  0.30965401,
        0.071924  , -0.00541399,  0.49063899,  0.29510101,  0.76056498,
       -0.19310001,  0.861871  ,  0.73134004,  0.889483  ,  0.73041601,
        0.6921    , -0.01101   , -0.17228799,  0.280772  ,  0.48982001,
       -0.29811899, -0.17408902,  0.806493  , -0.149249  , -0.221432  ,
       -0.13051602,  0.62860796,  0.35374498, -0.606888  ,  0.393793  ,
        0.102797  ,  0.92551997, -0.010614  ,  0.73485998, -0.35

In [ ]:
modelo.get_vector('unknown')

array([ 3.54566e-01, -5.48110e-02, -3.84400e-02,  1.24058e-01,
       -8.06360e-02,  5.91960e-02,  2.04200e-02, -1.78659e-01,
       -1.81335e-01, -9.11180e-02, -4.57600e-02, -1.06127e-01,
       -4.01736e-01,  4.19984e-01, -3.33250e-02,  1.77705e-01,
       -5.43387e-01, -8.51910e-02, -5.33234e-01, -2.86099e-01,
        3.33630e-02, -1.85811e-01,  6.87260e-02,  5.20180e-01,
       -2.98402e-01, -3.23700e-02, -2.72469e-01, -9.92270e-02,
        1.96912e-01, -1.82213e-01, -3.08074e-01, -3.69640e-02,
        9.03970e-02,  4.04880e-02, -6.46750e-02,  4.37740e-02,
        8.93440e-02, -4.98380e-02,  1.37664e-01, -1.27749e-01,
       -1.14070e-02, -1.04679e-01, -1.39191e-01, -5.21400e-03,
        2.90866e-01, -9.60230e-02,  2.38736e-01,  3.26503e-01,
        3.30080e-01,  2.03619e-01,  9.67800e-03,  1.19000e-01,
       -1.79310e-02,  1.29586e-01,  3.19442e-01, -1.05512e-01,
        9.31870e-02,  1.40411e-01,  5.84020e-02,  5.88410e-02,
        1.11957e-01,  4.73554e-01,  2.45136e-01, -5.096

### Vetorização dos Artigos

In [ ]:
# Treino
len(artigo_treino)

90000

In [ ]:
# Treino
len(artigo_teste)

20513

In [ ]:
# Função de vetorização de todos os textos
def matriz_vetores(textos, word2vec_model):
  x = len(textos)
  y = 300  # representação Word2Vec
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras_numeros = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_soma(palavras_numeros, word2vec_model=word2vec_model)

  return matriz

In [ ]:
# Vetorização dos títulos
matriz_vetores_treino = matriz_vetores(artigo_treino['title'], modelo)
matriz_vetores_teste = matriz_vetores(artigo_teste['title'], modelo)

In [ ]:
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape)

(90000, 300)
(20513, 300)


### Classificação

* Classificação multinomial

SAIBA MAIS

* [Regressão Logística](https://www.youtube.com/watch?v=yIYKR4sgzI8)
* 

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(max_iter=200)
LR.fit(matriz_vetores_treino, artigo_treino['category'])

LogisticRegression(max_iter=200)

In [ ]:
# Quantas iterações foram necessárias para convergir?
LR.n_iter_

array([121], dtype=int32)

### Avaliação do modelo

In [ ]:
# Acurácia do treino
LR.score(matriz_vetores_treino, artigo_treino['category'])

0.8164888888888889

In [ ]:
# Acurácia do teste
LR.score(matriz_vetores_teste, artigo_teste['category'])

0.7978842685126505

In [ ]:
# Predição nos testes
LR.predict(matriz_vetores_teste)

array(['colunas', 'cotidiano', 'esporte', ..., 'mercado', 'colunas',
       'mundo'], dtype=object)

In [ ]:
artigo_treino['category'].unique()

array(['mundo', 'cotidiano', 'mercado', 'esporte', 'ilustrada', 'colunas'],
      dtype=object)

#### Interpretando a classificação: Avaliação por categoria

**Precisão (precision)** mostra a quantidade de classes classificadas corretamente sobre tudo que foi classificado como aquela classe, ou seja, é a quantidade de verdadeiro-positivo sobre a soma verdadeiro-positivo + falso-positivo. 

Já a **revogação (recall)** é tudo que foi classificado corretamente sobre o total daquela classe, ou seja, é a quantidade de verdadeiro-positivo sobre verdadeiro-positivo + falso-negativos. Então, revogação mostra a capacidade de recuperar classes relevantes, enquanto precisão mostra o quão bem estas classes são recuperadas.


> Na matemática, a média harmônica geralmente é utilizada em situações em que a média de taxas é desejada. Ver vídeo [sobre média harmônica](https://www.youtube.com/watch?v=bamPhPrZk6w).

* [Conteúdo sobre as métricas](https://scikit-learn.org/stable/modules/model_evaluation.html#model-evaluation)

In [ ]:
from sklearn.metrics import classification_report


CR = classification_report(
    y_true=artigo_teste['category'], 
    y_pred=LR.predict(matriz_vetores_teste))
print(CR)

              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.61      0.80      0.69      1698
     esporte       0.92      0.88      0.90      4663
   ilustrada       0.14      0.89      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.74      0.86      0.79      2051

    accuracy                           0.80     20513
   macro avg       0.69      0.82      0.70     20513
weighted avg       0.83      0.80      0.81     20513



### Baseline: Dummy Classifier

Como saber se o nosso modelo é bom ou não? Não temos nenhuma referência para comparar.

[sklearn.dummy.DummyClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html)

In [ ]:
from sklearn.dummy import DummyClassifier

DC = DummyClassifier(strategy='stratified')
DC.fit(matriz_vetores_treino, artigo_treino['category'])
label_previsao_dc = DC.predict(matriz_vetores_teste)

CR_dummy = classification_report(
    y_true=artigo_teste['category'], 
    y_pred=label_previsao_dc)
print(CR_dummy)

              precision    recall  f1-score   support

     colunas       0.30      0.17      0.21      6103
   cotidiano       0.08      0.15      0.10      1698
     esporte       0.23      0.18      0.20      4663
   ilustrada       0.01      0.21      0.02       131
     mercado       0.29      0.17      0.21      5867
       mundo       0.10      0.17      0.13      2051

    accuracy                           0.17     20513
   macro avg       0.17      0.17      0.14     20513
weighted avg       0.24      0.17      0.19     20513



## Skip-Gram

In [ ]:
uri_skipgram_300 = 'https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/skip_s300.zip'

!wget $uri_skipgram_300
!unzip skip_s300.zip



Archive:  skip_s300.zip
  inflating: skip_s300.txt           


In [ ]:
# Leitura do arquivo de tokens
path_skipgram_300 = '/content/skip_s300.txt'

with open(path_skipgram_300) as file:
  for linha in range(5):
    print(next(file))

929606 300

</s> -0.001667 -0.000158 -0.000026 0.001300 -0.000796 0.001527 0.000046 0.000584 0.000449 -0.000100 0.000353 0.001251 0.001069 0.000506 0.000574 0.000838 -0.000930 -0.001220 0.000317 0.001315 -0.001120 0.001373 -0.000040 -0.001580 0.000421 -0.000667 -0.001556 -0.000746 0.001604 0.001157 -0.000027 0.000354 0.000358 -0.000527 -0.000573 -0.001512 -0.001557 -0.001637 0.001617 -0.001511 -0.001022 -0.001426 0.001086 -0.001033 0.000593 0.000724 0.000627 -0.000450 -0.001140 0.000333 0.000524 0.001541 0.000284 0.000617 -0.000807 -0.000088 -0.000364 0.001126 -0.001230 -0.001138 -0.001280 0.001330 0.001257 0.000576 0.000764 0.000684 0.001008 -0.000215 -0.000629 -0.001228 -0.001557 -0.000311 -0.000246 0.000045 0.001136 -0.000645 -0.000549 0.001099 0.000858 -0.000886 0.000553 0.000303 0.001433 0.000732 0.001321 -0.000894 -0.000700 -0.000661 -0.001484 -0.000950 -0.001556 -0.000809 0.000348 -0.000068 0.000724 -0.000569 -0.000161 -0.001628 -0.001437 -0.000259 -0.000296 -0.001571 0.000149 0

In [ ]:
# Construção do modelo Word2Vec-SkipGram a partir do texto ~2 min 33 s
from gensim.models import KeyedVectors

wv_from_text = KeyedVectors.load_word2vec_format(path_skipgram_300)
skipgram_model = wv_from_text

In [ ]:
palavras_numeros = tokenizador('texto 55 email@meu.com textosss')
vetor_texto = combinacao_de_vetores_soma(palavras_numeros, word2vec_model=skipgram_model)
vetor_texto

array([-6.68270012e-01, -2.79682003e-01,  3.50800250e-03,  2.45561004e-01,
       -7.74090067e-02,  7.85348007e-01, -4.56686035e-01,  5.67921989e-01,
       -4.25287008e-01, -1.08889401e+00, -9.01659951e-02, -4.68767993e-01,
       -4.95117024e-01, -3.37766021e-01,  7.96551973e-01, -2.32274000e-01,
        6.96054988e-01, -8.18373997e-01, -6.66531023e-01,  5.30870091e-02,
        3.87660000e-01, -1.91991397e+00, -1.18393011e-01,  4.95805996e-01,
        1.83799993e-01, -4.68564015e-01,  5.28749879e-02,  5.09001002e-01,
       -1.78386799e+00, -1.68957004e-01,  2.13609990e-02,  2.28826996e-01,
       -1.56790055e-02,  3.35800001e-01, -7.49274001e-01, -3.33013989e-01,
        3.38662986e-01, -4.37913019e-01,  1.16489802e+00, -3.30520123e-02,
        1.41941905e+00, -2.18389998e-02,  1.01358299e+00,  1.01134002e-01,
        6.17253039e-01, -3.40229003e-01,  9.48933031e-01,  4.34850985e-01,
        1.26273605e+00,  1.73449401e+00,  1.76766004e-01, -5.97031016e-01,
        1.08357996e-01,  

### Modelo

In [ ]:
# Construção da Matriz dos vetores
matriz_vetores_sg_treino = matriz_vetores(
    artigo_treino['title'], 
    word2vec_model=skipgram_model)

matriz_vetores_sg_teste = matriz_vetores(
    artigo_teste['title'], 
    word2vec_model=skipgram_model)

# Previsões
LR = LogisticRegression(max_iter=300)
LR.fit(X=matriz_vetores_sg_treino, 
       y=artigo_treino['category'])
print(LR.n_iter_)

[238]


In [ ]:
# Acurácia do treino
acuracia_treino = LR.score(matriz_vetores_sg_treino, artigo_treino['category'])
print(f'A acurácia do treino foi de {acuracia_treino:.2f}')

# Acurácia do teste
acuracia_teste = LR.score(matriz_vetores_sg_teste, artigo_teste['category'])
print(f'A acurácia do treino foi de {acuracia_teste:.2f}')

CR_skipgram = classification_report(
    y_true=artigo_teste['category'], 
    y_pred=LR.predict(matriz_vetores_sg_teste))
print(CR_skipgram)

print(CR)

A acurácia do treino foi de 0.83
A acurácia do treino foi de 0.81
              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.63      0.81      0.71      1698
     esporte       0.93      0.89      0.91      4663
   ilustrada       0.15      0.91      0.26       131
     mercado       0.85      0.82      0.83      5867
       mundo       0.76      0.86      0.81      2051

    accuracy                           0.81     20513
   macro avg       0.70      0.83      0.72     20513
weighted avg       0.84      0.81      0.82     20513

              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.61      0.80      0.69      1698
     esporte       0.92      0.88      0.90      4663
   ilustrada       0.14      0.89      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.74      0.86      0.79      2051

    accurac